In [7]:
import modin.pandas as mipd
import os
import time
import pandas as pd
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

import gc

In [35]:
class EmailNetwork:
    def __init__(self, e_path, allinonefile):
        df = pd.read_csv(allinonefile)
        self.e_path = e_path
        self.emailID_to_author = df[['message_id', 'dealised_author_full_name']].set_index('message_id').T.to_dict('index')
        print("Dict of email IDs build successfully")
        #print(list(self.emailID_to_author.items())[:10])
        del df
        gc.collect()

    def emailscreening(self, df_emails):
        in_emails= []
        df = df_emails
        df.query('is_bot == False', inplace=True)
        df = df[df['dealised_author_full_name'].notna()]
        
        for index, row in df.iterrows():
            message_id = row['message_id'].strip()
            # print(row['dealised_author_full_name'])
            sender_name = row['dealised_author_full_name']
            emailID_to_author[message_id] = sender_name

        # raise KeyError
        for index, row in df.iterrows():
            message_id = row['message_id']
            references = row['references']
            sender_name = row['dealised_author_full_name']

            # ignores if this email does not to previous emails
            if pd.isna(references) or references == 'None' or references == 'none':
                continue

            references = [r.strip() for r in references.replace('\n', ' ').replace('\t', ' ').split(' ') if r.strip()]

            # deal with the issue that a line breaker exists in message_id:
            # e.g., <4\n829AB62.6000302@apache.org>
            new_refs = set()
            for i in range(len(references)-1):
                if '<' in references[i] and '>' not in references[i] and '<' not in references[i+1] and '>' in references[i+1]:
                    new_refs.add(references[i] + references[i+1])
            for r in references:
                if '<' in r and '>' in r:
                    new_refs.add(r)

            references = new_refs
            for reference_id in references:
                if reference_id not in emailID_to_author:
                    continue
                else:
                    in_emails.append(row)
                    break


        return pd.DataFrame(in_emails)

    def onedfscreen(self, emailfile):
        this_df = pd.read_csv(self.e_path+emailfile)
        screened_df= self.emailscreening(this_df)
        return screened_df


In [23]:
e_path = '../monthly_data/emails/'
projects = os.listdir(e_path)
projects.sort(key= lambda x:int(x.split('__')[1].split('.')[0]))

screener = EmailNetwork(e_path= e_path, allinonefile= "../email_check/emails_preprocessed.csv")

for project in tqdm(projects[:5]):

    social_net = {}
    emailID_to_author = {}
    project_name, period = project.replace('.csv', '').split('__')

    this_df = pd.read_csv(e_path+project)
    screened_df= screener.emailscreening(this_df)

screened_df


Dict of email IDs build successfully
[('<AANLkTi=JbHG9LyB1rDa5xukLrr9cnmRQpLLb8isr=4hS@mail.gmail.com>', ['Ankur Garg']), ('<108351.8957.qm@web27805.mail.ukl.yahoo.com>', ['Mark Struberg']), ('<AANLkTi=zcdCDNo=c03My_MKbuXCzcku+6jUeq90gFwd8@mail.gmail.com>', ['Ankur Garg']), ('<AANLkTim1muzKJ_gB0p9HDKyXYScSBTbvQAhspHMUUMON@mail.gmail.com>', ['Jeremy Bauer']), ('<AANLkTimEhQxtYhwYEFMkwgk17JtYSOcwv2JzUU4bk-TX@mail.gmail.com>', ['Brian Maso']), ('<CAE9L6G0A0RZjiFnNPrRmh+bBCB9QU8oOfmHZ517BeTke85xbgA@mail.gmail.com>', ['Matt Benson']), ('<AANLkTimufZBOXn4P-mJuGg=9q=skj1kaW29r5U-t4SBZ@mail.gmail.com>', ['Gerhard Petracek']), ('<162205.88305.qm@web27802.mail.ukl.yahoo.com>', ['Mark Struberg']), ('<BANLkTi=0mG3jco43u=tYVnmiBiHj4nHDyQ@mail.gmail.com>', ['Matt Benson']), ('<4DBA2AD9.4040907@apache.org>', ['Donald Woods'])]


100%|██████████| 5/5 [00:00<00:00, 105.62it/s]


,project_name,list_name,date,month,message_id,sender_name,references,sender_email,in_reply_to,cc_list,receiver_email,subject,body,from_commit,author_full_name,is_bot,dealised_author_full_name
1,npanday,dev,2010-08-14 10:50:59,0.0,<AANLkTimLKVo+mHsySsMjcm=xD2nFfWruj_+x0Crys1dQ...,Deng Ching,<2651AC78-8AEE-44DA-BDF4-467CE56CDFBD@apache.o...,oching@apache.org,<F6BA368E-9599-44FD-81CF-ACC3FF8485B7@apache.org>,NaN,Brett Porter <brett@apache.org>,Re: New mailing lists to use!,"Please add me as mod too :)\n\nThanks,\nDeng\n...",False,deng ching,False,Deng Chingmallete
2,npanday,dev,2010-08-14 15:29:33,0.0,<3288FAD7-6D60-4E5C-8526-66C1D3CB6C49@apache.org>,Brett Porter,<2651AC78-8AEE-44DA-BDF4-467CE56CDFBD@apache.o...,brett@apache.org,<AANLkTimLKVo+mHsySsMjcm=xD2nFfWruj_+x0Crys1dQ...,NaN,Deng Ching <oching@apache.org>,Re: New mailing lists to use!,"Aded!\n\nOn 14/08/2010, at 12:50 PM, Deng Chin...",False,brett porter,False,Brett Porter
2,npanday,dev,2010-08-14 15:29:33,0.0,<3288FAD7-6D60-4E5C-8526-66C1D3CB6C49@apache.org>,Brett Porter,<2651AC78-8AEE-44DA-BDF4-467CE56CDFBD@apache.o...,brett@apache.org,<AANLkTimLKVo+mHsySsMjcm=xD2nFfWruj_+x0Crys1dQ...,NaN,Deng Ching <oching@apache.org>,Re: New mailing lists to use!,"Aded!\n\nOn 14/08/2010, at 12:50 PM, Deng Chin...",False,brett porter,False,Brett Porter
5,npanday,dev,2010-08-17 20:18:56,0.0,<5F9954A6-A4C0-463D-BB3E-49AABD33F0B2@lcorneli...,Lars Corneliussen,<4F71A5F0-051E-4469-A061-C3C4A1E82D91@apache.org>,me@lcorneliussen.de,<4F71A5F0-051E-4469-A061-C3C4A1E82D91@apache.org>,NaN,"""npanday-dev@incubator.apache.org"" <npanday-de...",Re: Code import,Sometimes I missed the history from NMaven. So...,False,lars corneliussen,False,Lars Corneliussen
7,npanday,dev,2010-08-19 09:44:45,0.0,<4C6C8C8D.7070005@g2ix.net>,Liit Padilla,<1153575176.1172.1282181958877.JavaMail.root@c...,apadilla@g2ix.net,<1153575176.1172.1282181958877.JavaMail.root@c...,NaN,npanday-dev@incubator.apache.org,Re: Code import,"+ 1 on option 3\n\nthanks,\n\nliit\n\nOn 8/19/...",False,liit padilla,False,Liit Padilla
9,npanday,dev,2010-08-20 11:04:43,0.0,<0A9FE1CC-4678-4BBD-BED5-54178EA2C68A@apache.org>,Brett Porter,<1305535226.2200.1282265750828.JavaMail.root@c...,brett@apache.org,<1305535226.2200.1282265750828.JavaMail.root@c...,NaN,npanday-dev@incubator.apache.org,Re: [Apache] User Details,"I've added them for Dennis, Gav, Deng, Wendy a...",False,brett porter,False,Brett Porter
11,npanday,dev,2010-08-20 06:45:12,0.0,<4C6E0858.2060502@lcorneliussen.de>,Lars Corneliussen,<628946796.2214.1282266709303.JavaMail.root@ce...,me@lcorneliussen.de,<628946796.2214.1282266709303.JavaMail.root@ce...,NaN,npanday-dev@incubator.apache.org,Re: [Apache] User Details,"Hello,\n\nmy username is 'lcorneliussen'. Can ...",False,lars corneliussen,False,Lars Corneliussen
13,npanday,dev,2010-08-20 13:23:12,0.0,<4C6E1140.4010909@maestrodev.com>,Liit Padilla,<1752928777.2485.1282280369784.JavaMail.root@c...,apadilla@maestrodev.com,<1752928777.2485.1282280369784.JavaMail.root@c...,NaN,npanday-dev@incubator.apache.org,Re: [Apache] User Details,"Hi Joe,\n\nMy username is apadilla.\n\n\nthank...",False,liit padilla,False,Liit Padilla
15,npanday,dev,2010-08-31 12:17:23,0.0,<44AC6C13-1305-4D0F-92E3-A1430A15A09B@apache.org>,Brett Porter,<1153575176.1172.1282181958877.JavaMail.root@c...,brett@apache.org,<4C6C8C8D.7070005@g2ix.net>,NaN,npanday-dev@incubator.apache.org,Re: Code import,I'm making progress here slowly. It will be re...,False,brett porter,False,Brett Porter
15,npanday,dev,2010-08-31 12:17:23,0.0,<44AC6C13-1305-4D0F-92E3-A1430A15A09B@apache.org>,Brett Porter,<1153575176.1172.1282181958877.JavaMail.root@c...,brett@apache.org,<4C6C8C8D.7070005@g2ix.net>,NaN,npanday-dev@incubator.apache.org,Re: Code import,I'm making progress here slowly. It will be re...,False,brett porter,False,Brett Porter


In [36]:
from multiprocessing import Pool

if __name__ == '__main__' :
    print("Build Screener")
    e_path = '../monthly_data/emails/'
    projects = os.listdir(e_path)
    projects.sort(key= lambda x:int(x.split('__')[1].split('.')[0]))

    screener = EmailNetwork(e_path= e_path, allinonefile= "../email_check/emails_preprocessed.csv")

    with Pool(10) as p:
        result = list(tqdm(p.imap(screener.onedfscreen, projects, chunksize= 12), total=len(projects), desc="Multiprocess preprocessing Screening"))

    p.close()
    p.join()

    df_result = pd.concat(result, axis= 0, ignore_index= True)
    

Build Screener


Dict of email IDs build successfully


Multiprocess preprocessing Screening: 100%|██████████| 6357/6357 [05:22<00:00, 19.72it/s]


In [38]:
df = df_result
df['body']= df['body'].apply(lambda x: None if x =='none' else x)
df = df.where(df['body'].isnull()==False).dropna(how= 'all')

In [45]:
len(df_result)

642845

In [44]:
df_result.to_csv("./edges_emails.csv", index= None)

In [40]:
df_result.iloc[5].body

"I think next question is what repositories this project wants. I see that\nhttps://github.com/datasketches has 17 repositories. As far as I know there\nis no restriction in ASF about how many repositories to have.\n\nAnother question is whether to use ASF's Jira installation or something\nelse (like GitHub issues).\n\nKenn\n\nOn Sun, Apr 21, 2019 at 7:52 PM Kenneth Knowles <klk@google.com> wrote:\n\n> I have removed private@datasketches.apache.org since this is not a\n> private matter. That list is generally for things like discussing other\n> people as potential committers or PMC members, security vulnerabilities,\n> and occasionally brand use questions.\n>\n> I recommend not cross-posting to private and public lists, to avoid anyone\n> accidentally sending a sensitive reply to a public list (which is archived\n> in perpetuity).\n>\n> Kenn\n>\n> On Sun, Apr 21, 2019 at 5:14 PM Liang Chen <chenliang613@apache.org>\n> wrote:\n>\n>> Hi\n>>\n>> Apart from the mailing list, how about othe

In [41]:
def onlyreply(txt):
    txt = str(txt)
    posi = txt.find("> wrote:\n")
    if posi == -1:
        return txt
    else:
        while txt[posi]!="\n" and posi>0:
            posi-= 1
        if posi>0:
            return txt[:posi]
        else:
            return txt 

In [42]:
onlyreply(df_result.iloc[5].body)

"I think next question is what repositories this project wants. I see that\nhttps://github.com/datasketches has 17 repositories. As far as I know there\nis no restriction in ASF about how many repositories to have.\n\nAnother question is whether to use ASF's Jira installation or something\nelse (like GitHub issues).\n\nKenn\n"

In [ ]:

for project in tqdm(projects):

    social_net = {}
    emailID_to_author = {}
    project_name, period = project.replace('.csv', '').split('__')

    fnames = [project_name+'__'+str(int(period)+i)+'.csv' for i in range(time_resolution) ]
    for fname in fnames:
        if not os.path.exists(e_path+fname):
            continue
        df = pd.read_csv(e_path+fname)
        df.query('is_bot == False', inplace=True)
        df = df[df['dealised_author_full_name'].notna()]
        
        for index, row in df.iterrows():
            message_id = row['message_id'].strip()
            # print(row['dealised_author_full_name'])
            sender_name = row['dealised_author_full_name']
            emailID_to_author[message_id] = sender_name

        # raise KeyError
        for index, row in df.iterrows():
            message_id = row['message_id']
            references = row['references']
            sender_name = row['dealised_author_full_name']

            # ignores if this email does not to previous emails
            if pd.isna(references) or references == 'None' or references == 'none':
                continue

            references = [r.strip() for r in references.replace('\n', ' ').replace('\t', ' ').split(' ') if r.strip()]

            # deal with the issue that a line breaker exists in message_id:
            # e.g., <4\n829AB62.6000302@apache.org>
            new_refs = set()
            for i in range(len(references)-1):
                if '<' in references[i] and '>' not in references[i] and '<' not in references[i+1] and '>' in references[i+1]:
                    new_refs.add(references[i] + references[i+1])
            for r in references:
                if '<' in r and '>' in r:
                    new_refs.add(r)

            references = new_refs
            for reference_id in references:
                if reference_id not in emailID_to_author:
                    continue
                prev_author = emailID_to_author[reference_id]
                # if it's the same person, continue
                if prev_author == sender_name:
                    continue
                if sender_name not in social_net:
                    social_net[sender_name] = {}
                if prev_author not in social_net:
                    social_net[prev_author] = {}

                # if node B replies node A, it means B sends signal to A
                if prev_author not in social_net[sender_name]:
                    social_net[sender_name][prev_author] = {}
                    social_net[sender_name][prev_author]['weight'] = 0
                social_net[sender_name][prev_author]['weight'] += 1

                # if node B replies node A, it means A also sent signal to B
                if sender_name not in social_net[prev_author]:
                    social_net[prev_author][sender_name] = {}
                    social_net[prev_author][sender_name]['weight'] = 0
                social_net[prev_author][sender_name]['weight'] += 1

    #save as directed graph
    g = nx.DiGraph(social_net)
    # add disconnected nodes
    g.add_nodes_from(social_net.keys())
    nx.write_edgelist(g, to_path + '{}__{}.edgelist'.format(project_name, str(period)), delimiter='##', data=["weight"])
    gx_socialnets.append(g)